## Process an External Video with mask
This script will start with a directory that contains an  
MP4 video file that we want to create a navigation heatmap for.  
  
The script requires a source_video to be defined.  
A temp conversion directory is created in the same directory as the video file.
The temp directory will be erased when the superimposed video is completed.  
A final superimposed video will be in the same location as the  
original MP4 file with the same name appended with '_heatmap'.

In [ ]:
import os

In [ ]:
# OPTIONAL SETTING
# Here we override the keras backend env variable to use plaidml
# plaidml will use a GPU
# This assignment needs to be added before loading keras libraries

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# to install plaidML, activate appropriate environment and then:
#   pip install -U plaidml-keras
#   plaidml-setup

In [ ]:
import cv2
import video_support_processes as pf_video

In [ ]:
source_video_filepath = '/Users/markmcdonald/Desktop/89FinalData/ext_train_data/mountain_biking/GOPR0145.MP4'

In [ ]:
# Create Temp Directory for processing extract JPG frames
source_video_path = os.path.dirname(source_video_filepath)
temp_dir = os.path.join(source_video_path,'images','data')

try:
    os.mkdir(os.path.join(source_video_path,'images'))  
except:
    try:
        os.mkdir(temp_dir)
        print("Success: Created Directory: ", temp_dir)
    except(FileExistsError):
        print("Success: Path Already Exists: ", temp_dir)


In [ ]:
def extractFrames(video_filepath, targetdir):
 
    cap = cv2.VideoCapture(video_filepath)
    fps = cap.get(cv2.CAP_PROP_FPS)  # determine the frames per second
    count = 0
 
    while (cap.isOpened()):
 
        # Capture frame-by-frame
        ret, frame = cap.read()
 
        if ret == True:
#             print('Read %d frame: ' % count, ret)
            cv2.imwrite(os.path.join(targetdir, "frame{:d}.jpg".format(count)), frame)  # save frame as JPEG file
            count += 1
        else:
            break
 
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    
    return fps

In [ ]:
# You only need to do this once per video
fps = extractFrames(source_video_filepath, temp_dir)

In [ ]:
# Loading a prevously saved pickle file
import pickle

def pickle_load(pickel_filepath):
    with open(pickel_filepath, 'rb') as f:
        data = pickle.load(f)
        return data

In [ ]:
# process multiple models on the same file
pickle_name = 'data.pickle'

pickel_filepaths = []
pickel_filepaths.append('/Users/markmcdonald/Desktop/89FinalData/results/1557741539_CNN_encoder2400_epochs=40_notes=dropout=.4_loss=0.562')
# multple filepaths can be added to create multuple videos
# pickel_filepaths.append('/Users/markmcdonald/Desktop/89FinalData/results/1557686089_CNN_encoder2400_epochs=70_notes=rot30-.6-1.0zoom_dropout=.5_loss=0.5247')



In [ ]:
cap = cv2.VideoCapture(source_video_filepath)
fps = cap.get(cv2.CAP_PROP_FPS)  # determine the frames per second
fps

In [ ]:
# use bigger batch size for more computer memory
for pickel_filepath in pickel_filepaths:
    data = pickle_load(os.path.join(pickel_filepath, pickle_name))
    description = data[0]
    model = data[1]
    pf_video.create_video_with_test_data(source_video_path, model, description, source_video_path, 
                                         batch_size=128, 
                                         fps=fps,
                                         invert=True)